<a href="https://colab.research.google.com/github/silver96/prj_10x10/blob/main/project_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

In [ ]:
# 모듈 임포트
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle # pickle을 이용하면 int 값을 그대로 저장하고 그대로 불러옴
from konlpy.tag import Okt # 형태소 분리기 임포트

ModuleNotFoundError: ignored

In [ ]:
pd.set_option('display.unicode.east_asian_width',True)

In [ ]:
df = pd.read_csv('/content/10x10(kor2).csv', index_col=0)

print(df.head())
print(df.info())

                                                          title   category
0  리빙숲 각도조절 가능 브리즈 사무실 발받침대 다리받침대......  Furniture
1           커블체어      원데이특가   무료배송까지              Furniture
2                 자취 꿀템 스누피 테이블 주말 특가              Furniture
3  스마트폰 태블릿  거치 인체공학 테이블 베드트레이   ......     Furniture
4                  모던데일  단 무빙 트롤리 수납선반             Furniture
<class 'pandas.core.frame.DataFrame'>
Int64Index: 103666 entries, 0 to 107699
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title     103666 non-null  object
 1   category  103666 non-null  object
dtypes: object(2)
memory usage: 2.4+ MB
None


In [ ]:
# 중복 행 확인
col_dup = df['title'].duplicated()
print(col_dup)

sum_dup = df.title.duplicated().sum()
print(sum_dup)

0         False
1         False
2         False
3         False
4         False
          ...  
107695    False
107696    False
107697    False
107698    False
107699    False
Name: title, Length: 103666, dtype: bool
13146


In [ ]:
# title 컬럼 기준으로 중복 제거 (row 통째 제거)
df = df.drop_duplicates(subset=['title'])

sum_dup = df.title.duplicated().sum()
print(sum_dup)

0


In [ ]:
# 인덱스 새로 고침
df.reset_index(drop=True, # False로 줄 시 기존 인덱스를 colum으로 올림
               inplace=True)

In [ ]:
print(df.head())
print(df.tail())


                                                          title   category
0  리빙숲 각도조절 가능 브리즈 사무실 발받침대 다리받침대......  Furniture
1           커블체어      원데이특가   무료배송까지              Furniture
2                 자취 꿀템 스누피 테이블 주말 특가              Furniture
3  스마트폰 태블릿  거치 인체공학 테이블 베드트레이   ......     Furniture
4                  모던데일  단 무빙 트롤리 수납선반             Furniture
                                                             title category
90515      센수 멘트 오픈링 패션 꼬임         검지 엄지             Jewelry
90516  센수 베니스           빈티지 반지        우정링 엄지 ......  Jewelry
90517  센수 메종드      실버  반지        엄지손가락 두줄 ......    Jewelry
90518  센수 메리얼 레이어드 두줄 데일리목걸이        여자 ......    Jewelry
90519     센수 리얼 송치 가죽         브레이슬릿 여성가             Jewelry


In [ ]:
# 데이터 나누기
X = df['title']
Y = df['category']

In [ ]:
# 원핫인코딩 위해 라벨로 변경
encoder = LabelEncoder()
labeled_Y = encoder.fit_transform(Y)
label = encoder.classes_

print(label)

['Baby,Kids' 'Beauty' 'Deco' 'FasihonClothing' 'FasionGoods' 'Food'
 'Furniture' 'Jewelry' 'Kitchen' 'Life']


In [ ]:
# 지정한 라벨 저장
# 이미 학습한 라벨들을 이용해야 학습된 결과들과 똑같이 도출

with open('/content/10x10_category_encoder.pickle', 'wb') as f:
    pickle.dump(encoder, f)   

In [ ]:
print(labeled_Y)

[6 6 6 ... 7 7 7]


In [ ]:
# 원핫인코딩 => 정답데이터 원핫인코딩(희소벡터처리)
onehot_Y = to_categorical(labeled_Y)

print(onehot_Y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [ ]:
# 첫번째 데이터 형태소로 분리
okt = Okt()
print(type(X))

okt_X = okt.morphs(X[0])
print(X[0])
print(okt_X)

<class 'pandas.core.series.Series'>
리빙숲 각도조절 가능 브리즈 사무실 발받침대 다리받침대
['리빙숲', '각도', '조절', '가능', '브리', '즈', '사무실', '발', '받침', '대', '다리', '받침', '대']


In [ ]:
# 모든 row를 형태소로 분리
for i in range(len(X)):
    X[i] = okt.morphs(X[i])
print(X)

0        [리빙숲, 각도, 조절, 가능, 브리, 즈, 사무실, 발, 받침, 대, 다리, 받침...
1         [커블, 체어, 원, 데, 이특, 가, 무료, 배송, 까지]
2               [자취, 꿀템, 스누피, 테이블, 주말, 특, 가]
3        [스마트폰, 태블릿, 거치, 인체, 공학, 테이블, 베드, 트, 레이]...
4               [모던, 데일, 단, 무빙, 트롤리, 수납, 선반]
                               ...                        
90515       [센수, 멘트, 오픈, 링, 패션, 꼬임, 검지, 엄지]
90516       [센수, 베니스, 빈티, 지, 반지, 우정, 링, 엄지]
90517       [센수, 메종드, 실버, 반지, 엄지손가락, 두, 줄]
90518    [센수, 메, 리얼, 레이어, 드, 두, 줄, 데, 일리, 목걸이, 여자]...
90519     [센수, 리얼, 송치, 가죽, 브레이, 슬릿, 여성, 가]
Name: title, Length: 90520, dtype: object


In [ ]:
# stopwords 불러오기
stopwords = pd.read_csv('/content/stopwords2.csv')

In [ ]:
# 추가한 stopwords 확인
print(stopwords.tail(40))

     Unnamed: 0      stopword
831         832          지샥
832         833          우레
833         834  블렌드바이오
834         835          블루
835         836          모음
836         837        부엉이
837         838          스틱
838         839            팩
839         840          롯데
840         841          지애
841         842            봉
842         843          블랙
843         844          레이
844         845          러리
845         846          당일
846         847          발송
847         848          순수
848         849          핑크
849         850          쓱싹
850         851          리사
851         852          감사
852         853        스누피
853         854        화이트
854         855          송월
855         856       그레이 
856         857            여
857         858      빔보빔바
858         859        스마일
859         860          쿠룬
860         861          스톤
861         862          데이
862         863          로우
863         864          백색
864         865          나그
865         866        겟잇미

In [ ]:
# stopwords에 있는 단어를 제거하고 리스트에 담기
words = [] # 빈 리스트 생성

for word in okt_X:
    if word not in list(stopwords['stopword']):
        words.append(word)
print(words)

['리빙숲', '각도', '조절', '가능', '브리', '즈', '사무실', '발', '받침', '대', '다리', '받침', '대']


In [ ]:
# 
for i in range(len(X)): # X=df['title']
    result = []
    for j in range(len(X[i])):
        if len(X[i][j]) > 1:
            if X[i][j] not in list(stopwords['stopword']):
                result.append(X[i][j])
    X[i] = ' '.join(result)
print(X)

0        리빙숲 각도 조절 가능 브리 사무실 받침 다리 받침
1                                     커블 체어 이특 무료
2                                   자취 꿀템 테이블 주말
3              스마트폰 태블릿 거치 인체 공학 테이블 베드
4                         모던 데일 무빙 트롤리 수납 선반
                               ...                       
90515                  센수 멘트 오픈 패션 꼬임 검지 엄지
90516                          센수 베니스 반지 우정 엄지
90517                         센수 메종드 반지 엄지손가락
90518                             센수 리얼 레이어 목걸이
90519                     센수 리얼 송치 가죽 브레이 슬릿
Name: title, Length: 90520, dtype: object


In [ ]:
#단어를 컴퓨터에서 인식시키기 위해 각 단어마다 숫자(라벨링) 부여 => 토큰화
# 수치적인 의미는 없음
token = Tokenizer()
token.fit_on_texts(X)
tokened_X = token.texts_to_sequences(X)
print(tokened_X[0])

[12173, 2166, 455, 1224, 1257, 716, 127, 872, 127]


In [ ]:
import pickle
#읽고 쓰는 과정에서 데이터의 자료형을 따로 인코딩할 필요 없이 그대로 저장

with open('/content/10x10_token3.pickle', 'wb') as f:
    pickle.dump(token, f)

In [ ]:
wordsize = len(token.word_index) + 1
print(wordsize)

20320


In [ ]:
max = 0
for i in range(len(tokened_X)):
    if max < len(tokened_X[i]):
        max = len(tokened_X[i])
print(max)

13


In [ ]:
#LSTM모델 사용을 위해 데이터 크기가 다른 것들을 0을 채워넣어 max사이즈로 통일시킴
X_pad = pad_sequences(tokened_X, max)
print(X_pad[:10])

[[    0     0     0     0 12173  2166   455  1224  1257   716   127   872
    127]
 [    0     0     0     0     0     0     0     0     0  6965    22 12174
   1654]
 [    0     0     0     0     0     0     0     0     0  3293 12175     8
   8001]
 [    0     0     0     0     0     0  1961  5217   227  5645  9467     8
   1116]
 [    0     0     0     0     0     0     0    90  1063  3585  2002    20
     54]
 [    0     0     0     0     0     0     0     0     0  4246   896   168
    302]
 [    0     0     0     0     0     0  4247   157   177    43    31  2112
    273]
 [    0     0     0     0     0     0     0  1064    31 12176  1885   243
   1064]
 [    0     0     0     0     0     0     0     0     0     0    12   140
      8]
 [    0     0     0     0     0     0    77 12177 12178  9468    77   242
   1162]]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_pad, onehot_Y, test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(81468, 13)
(9052, 13)
(81468, 10)
(9052, 10)


In [ ]:
xy = X_train, X_test, Y_train, Y_test
np.save('/content/10x10_data_max_{}_size_{}_2'.format(max, wordsize), xy)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
